# 모델 모니터링

이 노트북은 프로덕션 환경에서 머신러닝 모델의 성능을 모니터링하는 방법을 보여줍니다.

## 학습 내용
- 모델 성능 메트릭 추적
- 데이터 드리프트 감지
- 예측 분포 모니터링
- 알림 및 대시보드

In [ ]:
# 필요한 라이브러리 설치
!pip install -q scikit-learn pandas numpy matplotlib seaborn scipy

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 플롯 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. 기준 모델 학습

In [ ]:
# 데이터 로드
iris = load_iris()
X = iris.data
y = iris.target

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 초기 성능 측정
y_pred = model.predict(X_test_scaled)
baseline_accuracy = accuracy_score(y_test, y_pred)
baseline_precision = precision_score(y_test, y_pred, average='weighted')
baseline_recall = recall_score(y_test, y_pred, average='weighted')
baseline_f1 = f1_score(y_test, y_pred, average='weighted')

print("기준 모델 성능:")
print(f"Accuracy: {baseline_accuracy:.4f}")
print(f"Precision: {baseline_precision:.4f}")
print(f"Recall: {baseline_recall:.4f}")
print(f"F1 Score: {baseline_f1:.4f}")

## 2. 시간에 따른 성능 모니터링 시뮬레이션

In [ ]:
# 시간에 따른 성능 변화 시뮬레이션
np.random.seed(42)
n_periods = 30  # 30일

# 메트릭 저장
performance_history = {
    'day': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': [],
    'prediction_count': []
}

for day in range(1, n_periods + 1):
    # 데이터 드리프트 시뮬레이션 (15일 이후 성능 저하)
    if day > 15:
        # 노이즈 추가로 성능 저하 시뮬레이션
        noise = np.random.normal(0, 0.3, X_test_scaled.shape)
        X_test_noisy = X_test_scaled + noise * (day - 15) / 15
    else:
        X_test_noisy = X_test_scaled
    
    # 예측
    y_pred_day = model.predict(X_test_noisy)
    
    # 메트릭 계산
    acc = accuracy_score(y_test, y_pred_day)
    prec = precision_score(y_test, y_pred_day, average='weighted', zero_division=0)
    rec = recall_score(y_test, y_pred_day, average='weighted')
    f1 = f1_score(y_test, y_pred_day, average='weighted')
    
    # 기록
    performance_history['day'].append(day)
    performance_history['accuracy'].append(acc)
    performance_history['precision'].append(prec)
    performance_history['recall'].append(rec)
    performance_history['f1_score'].append(f1)
    performance_history['prediction_count'].append(len(y_test))

# 데이터프레임 생성
perf_df = pd.DataFrame(performance_history)
print("성능 히스토리:")
print(perf_df.head(10))

In [ ]:
# 성능 메트릭 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Model Performance Over Time', fontsize=16)

# Accuracy
axes[0, 0].plot(perf_df['day'], perf_df['accuracy'], marker='o')
axes[0, 0].axhline(y=baseline_accuracy, color='r', linestyle='--', label='Baseline')
axes[0, 0].axhline(y=baseline_accuracy * 0.95, color='orange', linestyle='--', label='Alert Threshold (95%)')
axes[0, 0].set_xlabel('Day')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Accuracy Over Time')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Precision
axes[0, 1].plot(perf_df['day'], perf_df['precision'], marker='o', color='green')
axes[0, 1].axhline(y=baseline_precision, color='r', linestyle='--', label='Baseline')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Precision')
axes[0, 1].set_title('Precision Over Time')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Recall
axes[1, 0].plot(perf_df['day'], perf_df['recall'], marker='o', color='purple')
axes[1, 0].axhline(y=baseline_recall, color='r', linestyle='--', label='Baseline')
axes[1, 0].set_xlabel('Day')
axes[1, 0].set_ylabel('Recall')
axes[1, 0].set_title('Recall Over Time')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# F1 Score
axes[1, 1].plot(perf_df['day'], perf_df['f1_score'], marker='o', color='red')
axes[1, 1].axhline(y=baseline_f1, color='r', linestyle='--', label='Baseline')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('F1 Score')
axes[1, 1].set_title('F1 Score Over Time')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 데이터 드리프트 감지

In [ ]:
def detect_drift_ks_test(reference_data, current_data, feature_names, threshold=0.05):
    """
    Kolmogorov-Smirnov 테스트를 사용한 데이터 드리프트 감지
    """
    drift_detected = {}
    
    for i, feature_name in enumerate(feature_names):
        # KS 테스트
        statistic, pvalue = stats.ks_2samp(
            reference_data[:, i],
            current_data[:, i]
        )
        
        # 드리프트 감지 (p-value가 threshold보다 작으면 드리프트)
        is_drift = pvalue < threshold
        
        drift_detected[feature_name] = {
            'drift': is_drift,
            'statistic': statistic,
            'pvalue': pvalue
        }
    
    return drift_detected

# 드리프트가 있는 데이터 생성 (노이즈 추가)
X_drifted = X_test + np.random.normal(0, 0.5, X_test.shape)

# 드리프트 감지
drift_results = detect_drift_ks_test(
    X_train,
    X_drifted,
    iris.feature_names
)

print("데이터 드리프트 감지 결과:")
print("=" * 60)
for feature, result in drift_results.items():
    status = "⚠️ DRIFT DETECTED" if result['drift'] else "✓ No drift"
    print(f"{feature}:")
    print(f"  Status: {status}")
    print(f"  P-value: {result['pvalue']:.4f}")
    print(f"  Statistic: {result['statistic']:.4f}")
    print()

In [ ]:
# 특성 분포 비교 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Distribution Comparison (Train vs Drifted)', fontsize=16)

for idx, (ax, feature_name) in enumerate(zip(axes.flatten(), iris.feature_names)):
    # 히스토그램
    ax.hist(X_train[:, idx], bins=20, alpha=0.5, label='Training Data', density=True)
    ax.hist(X_drifted[:, idx], bins=20, alpha=0.5, label='Current Data', density=True)
    
    # 드리프트 상태 표시
    drift_status = "DRIFT" if drift_results[feature_name]['drift'] else "OK"
    ax.set_title(f'{feature_name} [{drift_status}]')
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 예측 분포 모니터링

In [ ]:
# 시간에 따른 예측 분포 시뮬레이션
prediction_history = []

for day in range(1, 31):
    # 데이터 변화 시뮬레이션
    if day > 15:
        noise = np.random.normal(0, 0.3, X_test_scaled.shape)
        X_current = X_test_scaled + noise * (day - 15) / 15
    else:
        X_current = X_test_scaled
    
    # 예측
    predictions = model.predict(X_current)
    
    # 각 클래스의 예측 비율
    for pred_class in range(3):
        count = np.sum(predictions == pred_class)
        prediction_history.append({
            'day': day,
            'class': iris.target_names[pred_class],
            'count': count,
            'percentage': count / len(predictions) * 100
        })

pred_df = pd.DataFrame(prediction_history)

# 예측 분포 시각화
plt.figure(figsize=(15, 6))

for class_name in iris.target_names:
    class_data = pred_df[pred_df['class'] == class_name]
    plt.plot(class_data['day'], class_data['percentage'], 
             marker='o', label=class_name)

plt.xlabel('Day')
plt.ylabel('Prediction Percentage (%)')
plt.title('Prediction Distribution Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. 알림 시스템

In [ ]:
def check_model_health(current_metrics, baseline_metrics, thresholds):
    """
    모델 상태 확인 및 알림 생성
    """
    alerts = []
    
    for metric_name, current_value in current_metrics.items():
        baseline_value = baseline_metrics.get(metric_name)
        threshold = thresholds.get(metric_name, 0.95)
        
        if baseline_value is not None:
            # 임계값 확인
            if current_value < baseline_value * threshold:
                degradation = ((baseline_value - current_value) / baseline_value) * 100
                alerts.append({
                    'metric': metric_name,
                    'severity': 'HIGH' if degradation > 10 else 'MEDIUM',
                    'message': f"{metric_name} degraded by {degradation:.2f}%",
                    'current': current_value,
                    'baseline': baseline_value
                })
    
    return alerts

# 기준 메트릭
baseline_metrics = {
    'accuracy': baseline_accuracy,
    'precision': baseline_precision,
    'recall': baseline_recall,
    'f1_score': baseline_f1
}

# 임계값 설정 (기준값의 95%)
thresholds = {
    'accuracy': 0.95,
    'precision': 0.95,
    'recall': 0.95,
    'f1_score': 0.95
}

# 최근 성능 (30일차)
recent_metrics = {
    'accuracy': perf_df.iloc[-1]['accuracy'],
    'precision': perf_df.iloc[-1]['precision'],
    'recall': perf_df.iloc[-1]['recall'],
    'f1_score': perf_df.iloc[-1]['f1_score']
}

# 상태 확인
alerts = check_model_health(recent_metrics, baseline_metrics, thresholds)

print("모델 상태 알림:")
print("=" * 70)
if alerts:
    for alert in alerts:
        print(f"\n[{alert['severity']}] {alert['message']}")
        print(f"  Current: {alert['current']:.4f}")
        print(f"  Baseline: {alert['baseline']:.4f}")
else:
    print("✓ 모든 메트릭이 정상 범위 내에 있습니다.")

## 6. 모니터링 대시보드 요약

In [ ]:
# 대시보드 요약 생성
def generate_monitoring_summary(perf_df, drift_results, alerts):
    summary = {
        'monitoring_period': f"{perf_df['day'].min()} - {perf_df['day'].max()} days",
        'total_predictions': perf_df['prediction_count'].sum(),
        'avg_accuracy': perf_df['accuracy'].mean(),
        'current_accuracy': perf_df.iloc[-1]['accuracy'],
        'accuracy_trend': 'decreasing' if perf_df['accuracy'].iloc[-1] < perf_df['accuracy'].iloc[0] else 'stable/increasing',
        'drift_detected': sum(1 for r in drift_results.values() if r['drift']),
        'total_features': len(drift_results),
        'active_alerts': len(alerts),
        'alert_severity': [a['severity'] for a in alerts]
    }
    return summary

summary = generate_monitoring_summary(perf_df, drift_results, alerts)

print("\n" + "=" * 70)
print("모니터링 대시보드 요약")
print("=" * 70)
print(f"모니터링 기간: {summary['monitoring_period']}")
print(f"총 예측 수: {summary['total_predictions']:,}")
print(f"평균 정확도: {summary['avg_accuracy']:.4f}")
print(f"현재 정확도: {summary['current_accuracy']:.4f}")
print(f"정확도 추세: {summary['accuracy_trend']}")
print(f"드리프트 감지: {summary['drift_detected']}/{summary['total_features']} features")
print(f"활성 알림: {summary['active_alerts']}")
if summary['active_alerts'] > 0:
    print(f"알림 심각도: {', '.join(summary['alert_severity'])}")
print("=" * 70)

## 요약

이 노트북에서는 프로덕션 환경의 모델 모니터링에 대해 학습했습니다:
1. ✅ 시간에 따른 모델 성능 추적
2. ✅ 데이터 드리프트 감지 (Kolmogorov-Smirnov 테스트)
3. ✅ 예측 분포 모니터링
4. ✅ 알림 시스템 구현
5. ✅ 모니터링 대시보드 요약

**모니터링 모범 사례:**
- 기준 메트릭 설정 및 추적
- 정기적인 데이터 드리프트 확인
- 성능 저하 시 자동 알림
- 예측 분포 변화 모니터링
- 정기적인 모델 재학습 고려

다음 단계: CI/CD 파이프라인 구축